<a href="https://colab.research.google.com/github/alpguimaraes/python_para_investimentos/blob/master/GammaFlip_interativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Rodar essa célula somente uma vez ###
# Delete a # na linha abaixo, execute e coloque de volta a #

#!pip install plotly

In [ ]:
import pandas as pd
import plotly
pd.set_option('plotting.backend','plotly')
import plotly.graph_objs as go
import numpy as np
import scipy
from scipy.stats import norm
#import matplotlib.pyplot as plt
import calendar
from datetime import datetime, timedelta, date

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format

In [ ]:
# Parametros de entrada
filename = 'ndx_quotedata.csv'

# Black-Scholes European-Options Gamma
def calcGammaEx(S, K, vol, T, r, q, optType, OI):
    if T == 0 or vol == 0:
        return 0

    dp = (np.log(S/K) + (r - q + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    dm = dp - vol*np.sqrt(T)

    if optType == 'call':
        gamma = np.exp(-q*T) * norm.pdf(dp) / (S * vol * np.sqrt(T))
        return OI * 100 * S * S * 0.01 * gamma
    else: # Gamma is same for calls and puts. This is just to cross-check
        gamma = K * np.exp(-r*T) * norm.pdf(dm) / (S * S * vol * np.sqrt(T))
        return OI * 100 * S * S * 0.01 * gamma

def isThirdFriday(d):
    return d.weekday() == 4 and 15 <= d.day <= 21

# Nova seção

In [ ]:
# Isso assume que o formato do arquivo CBOE não foi editado, ou seja, a tabela começa na linha 4
optionsFile = open(filename)
optionsFileData = optionsFile.readlines()
optionsFile.close()

In [ ]:
# Extraindo SPX spot
spotLine = optionsFileData[1]
spotPrice = float(spotLine.split('Last:')[1].split(',')[0])
fromStrike = 0.8 * spotPrice
toStrike = 1.2 * spotPrice

In [ ]:
# Extraindo a data de hoje
dateLine = optionsFileData[2]
todayDate = dateLine.split('Date: ')[1].split(',')
monthDay = todayDate[0].split(' ')

In [ ]:
if len(monthDay) == 2:
    year = int(monthDay[4])
    month = monthDay[2]
    day = int(monthDay[0])
else:
    if monthDay[2].isdigit():
        year = int(monthDay[4])
        month = monthDay[2]
        day = int(monthDay[0])
    else:
        year = int(monthDay[4])
        month = monthDay[2]
        day = int(monthDay[0])


# criar um dicionário para mapear os nomes dos meses em português para os equivalentes em inglês
nomes_meses = {'janeiro': 'January', 'fevereiro': 'February', 'março': 'March',
               'abril': 'April', 'maio': 'May', 'junho': 'June',
               'julho': 'July', 'agosto': 'August', 'setembro': 'September',
               'outubro': 'October', 'novembro': 'November', 'dezembro': 'December'}

# extrair o nome do mês da string de entrada
nome_mes_pt = monthDay[2]

# converter o nome do mês para inglês usando o dicionário
nome_mes_en = nomes_meses[nome_mes_pt]

# converter o nome do mês para o número correspondente (por exemplo, 'March' -> 3)
num_mes = datetime.strptime(nome_mes_en, '%B').month

# criar o objeto datetime
todayDate = datetime(year=year, month=num_mes, day=day)

In [ ]:
# create a dictionary to map Portuguese month names to English month names
month_names = {'janeiro': 'January', 'fevereiro': 'February', 'março': 'March',
               'abril': 'April', 'maio': 'May', 'junho': 'June',
               'julho': 'July', 'agosto': 'August', 'setembro': 'September',
               'outubro': 'October', 'novembro': 'November', 'dezembro': 'December'}

# extract the month name from the input string
month_name_pt = monthDay[2]

# convert the month name to English using the dictionary
month_name_en = month_names[month_name_pt]

# convert the month name to its corresponding number (e.g., 'March' -> 3)
month_number = datetime.strptime(month_name_en, '%B').month

# create the datetime object
todayDate = datetime(year=year, month=month_number, day=day)

In [ ]:
# Get SPX Options Data
df = pd.read_csv(filename, sep=",", header=None, skiprows=4)
df.columns = ['ExpirationDate','Calls','CallLastSale','CallNet','CallBid','CallAsk','CallVol',
              'CallIV','CallDelta','CallGamma','CallOpenInt','StrikePrice','Puts','PutLastSale',
              'PutNet','PutBid','PutAsk','PutVol','PutIV','PutDelta','PutGamma','PutOpenInt']


df['ExpirationDate'] = pd.to_datetime(df['ExpirationDate'], format='%a %b %d %Y')
df['ExpirationDate'] = df['ExpirationDate'] + timedelta(hours=16)
df['StrikePrice'] = df['StrikePrice'].astype(float)
df['CallIV'] = df['CallIV'].astype(float)
df['PutIV'] = df['PutIV'].astype(float)
df['CallGamma'] = df['CallGamma'].astype(float)
df['PutGamma'] = df['PutGamma'].astype(float)
df['CallOpenInt'] = df['CallOpenInt'].astype(float)
df['PutOpenInt'] = df['PutOpenInt'].astype(float)

In [ ]:
# ---=== CALCULATE SPOT GAMMA ===---
# Gamma Exposure = Unit Gamma * Open Interest * Contract Size * Spot Price
# To further convert into 'per 1% move' quantity, multiply by 1% of spotPrice
df['CallGEX'] = df['CallGamma'] * df['CallOpenInt'] * 100 * spotPrice * spotPrice * 0.01
df['PutGEX'] = df['PutGamma'] * df['PutOpenInt'] * 100 * spotPrice * spotPrice * 0.01 * -1

df['TotalGamma'] = (df.CallGEX + df.PutGEX) / 10**9
dfAgg = df.groupby(['StrikePrice']).sum(numeric_only=True)
strikes = dfAgg.index.values

In [ ]:
# Chart 1: Absolute Gamma Exposure
# define os dados
x_data = strikes
y_data = dfAgg['TotalGamma'].to_numpy()

# cria um gráfico de barras
fig = go.Figure(
    go.Bar(
        x=x_data,
        y=y_data,
        width=6,
        marker_color='rgb(26, 118, 255)',  # cor das barras
        marker_line_color='black',  # cor das linhas de contorno das barras
        marker_line_width=0.15,  # largura das linhas de contorno das barras
        name='Gamma Exposure'
    )
)

# adiciona uma linha vertical para o preço spot
fig.add_shape(
    type='line',
    x0=spotPrice,
    y0=min(y_data),
    x1=spotPrice,
    y1=max(y_data),
    line=dict(
        color='red',
        width=2,
        dash='dash'  # estilo da linha
    )
)

# define o layout do gráfico
fig.update_layout(
    title={
        'text': f"Total Gamma: ${df['TotalGamma'].sum():,.2f} Bn per 1% SPX Move",
        'font': {'size': 20, 'family': 'Arial Black',}
    },
    xaxis_title='Strike',
    yaxis_title='Spot Gamma Exposure ($ billions/1% move)',
    xaxis=dict(range=[fromStrike, toStrike]),
    yaxis=dict(tickformat='$,.2f'),
    plot_bgcolor='white',  # cor de fundo do gráfico
    font=dict(family='Arial', size=12, color='black')
)

# DEFINIR O TAMANHO DO GRÁFICO EM PIXELS
fig.update_layout(
    width=1750,
    height=800
)


# mostra o gráfico
fig.show()

In [ ]:
# Chart 2: Absolute Gamma Exposure by Calls and Puts
fig = go.Figure()
fig.add_bar(x=strikes, y=dfAgg['CallGEX'].to_numpy() / 10**9, width=6, name="Call Gamma")
fig.add_bar(x=strikes, y=dfAgg['PutGEX'].to_numpy() / 10**9, width=6, name="Put Gamma")
fig.update_xaxes(range=[fromStrike, toStrike])
chartTitle = "Total Gamma: $" + str("{:.2f}".format(df['TotalGamma'].sum())) + " Bn per 1% SPX Move"
fig.update_layout(title_text=chartTitle, title_font=dict(size=20, family="Arial Black"))
fig.update_xaxes(title_text="Strike")
fig.update_yaxes(title_text="Spot Gamma Exposure ($ billions/1% move)")
fig.add_shape(dict(type="line", x0=spotPrice, y0=0, x1=spotPrice, y1=max(dfAgg['CallGEX'].to_numpy() / 10**9), line=dict(color="black", width=2), name="SPX Spot:" + str("{:,.0f}".format(spotPrice))))

# DEFINIR O TAMANHO DO GRÁFICO EM PIXELS
fig.update_layout(
    width=1750,
    height=800
)

fig.show()


In [ ]:
# ---=== CALCULATE GAMMA PROFILE ===---
levels = np.linspace(fromStrike, toStrike, 60)

# For 0DTE options, I'm setting DTE = 1 day, otherwise they get excluded
df['daysTillExp'] = [1/262 if (np.busday_count(todayDate.date(), x.date())) == 0 \
                           else np.busday_count(todayDate.date(), x.date())/262 for x in df.ExpirationDate]

nextExpiry = df['ExpirationDate'].min()

df['IsThirdFriday'] = [isThirdFriday(x) for x in df.ExpirationDate]
thirdFridays = df.loc[df['IsThirdFriday'] == True]
nextMonthlyExp = thirdFridays['ExpirationDate'].min()

totalGamma = []
totalGammaExNext = []
totalGammaExFri = []

In [ ]:
# For each spot level, calc gamma exposure at that point
for level in levels:
    df['callGammaEx'] = df.apply(lambda row : calcGammaEx(level, row['StrikePrice'], row['CallIV'],
                                                          row['daysTillExp'], 0, 0, "call", row['CallOpenInt']), axis = 1)

    df['putGammaEx'] = df.apply(lambda row : calcGammaEx(level, row['StrikePrice'], row['PutIV'],
                                                         row['daysTillExp'], 0, 0, "put", row['PutOpenInt']), axis = 1)

    totalGamma.append(df['callGammaEx'].sum() - df['putGammaEx'].sum())

    exNxt = df.loc[df['ExpirationDate'] != nextExpiry]
    totalGammaExNext.append(exNxt['callGammaEx'].sum() - exNxt['putGammaEx'].sum())

    exFri = df.loc[df['ExpirationDate'] != nextMonthlyExp]
    totalGammaExFri.append(exFri['callGammaEx'].sum() - exFri['putGammaEx'].sum())

totalGamma = np.array(totalGamma) / 10**9
totalGammaExNext = np.array(totalGammaExNext) / 10**9
totalGammaExFri = np.array(totalGammaExFri) / 10**9

In [ ]:
# Find Gamma Flip Point
zeroCrossIdx = np.where(np.diff(np.sign(totalGamma)))[0]

negGamma = totalGamma[zeroCrossIdx]
posGamma = totalGamma[zeroCrossIdx+1]
negStrike = levels[zeroCrossIdx]
posStrike = levels[zeroCrossIdx+1]

zeroGamma = posStrike - ((posStrike - negStrike) * posGamma/(posGamma-negGamma))
zeroGamma = zeroGamma[0]

In [ ]:
# Chart 3: Gamma Exposure Profile
fig = go.Figure()

fig.add_trace(go.Scatter(x=levels, y=totalGamma, mode='lines', name='All Expiries'))
fig.add_trace(go.Scatter(x=levels, y=totalGammaExNext, mode='lines', name='Ex-Next Expiry'))
fig.add_trace(go.Scatter(x=levels, y=totalGammaExFri, mode='lines', name='Ex-Next Monthly Expiry'))

chartTitle = "Gamma Exposure Profile, SPX, " + todayDate.strftime('%d %b %Y')
fig.update_layout(title=chartTitle, xaxis_title='Index Price', yaxis_title='Gamma Exposure ($ billions/1% move)')
fig.update_layout(title_text=chartTitle, title_font=dict(size=20, family="Arial Black"))

fig.add_shape(
    dict(
        type="line",
        x0=spotPrice,
        y0=min(totalGamma),
        x1=spotPrice,
        y1=max(totalGamma),
        line=dict(color="red", width=1.5),
        name="SPX Spot: " + str("{:,.0f}".format(spotPrice))
    )
)

fig.add_shape(
    dict(
        type="line",
        x0=zeroGamma,
        y0=min(totalGamma),
        x1=zeroGamma,
        y1=max(totalGamma),
        line=dict(color="green", width=1.5),
        name="Gamma Flip: " + str("{:,.0f}".format(zeroGamma))
    )
)

fig.update_xaxes(range=[fromStrike, toStrike])
fig.update_yaxes(range=[min(totalGamma), max(totalGamma)])

fig.add_trace(
    go.Scatter(
        x=[fromStrike, zeroGamma, toStrike],
        y=[min(totalGamma), min(totalGamma), min(totalGamma)],
        mode="none",
        fill="toself",
        fillcolor="red",
        opacity=0.1,
        showlegend=False,
        name="Negative Gamma"
    )
)

fig.add_trace(
    go.Scatter(
        x=[fromStrike, zeroGamma, toStrike],
        y=[max(totalGamma), max(totalGamma), max(totalGamma)],
        mode="none",
        fill="toself",
        fillcolor="green",
        opacity=0.1,
        showlegend=False,
        name="Positive Gamma"
    )
)

# DEFINIR O TAMANHO DO GRÁFICO EM PIXELS
fig.update_layout(
    width=1400,
    height=700
)

fig.show()